In [52]:
import numpy as np

box1 = np.array([15, 10, 29, 35]) # x_l, y_l, x_r, y_r
box2 = np.array([20, 25, 45, 54]) # x_l, y_l, x_r, y_r
box3 = np.array([120, 25, 145, 54]) # x_l, y_l, x_r, y_r

boxes = [box1, box2]

def calculate_area(box):
  width = box[2]-box[0]
  height = box[3] - box[1]

  return width*height

def get_intersection(box1, box2):
  x_l = max(box1[0], box2[0])
  y_l = max(box1[1], box2[1])

  x_r = min(box1[2], box2[2])
  y_r = min(box1[3], box2[3])

  if x_r>=x_l and y_r>=y_l:

    intersecting_box = np.array([x_l, y_l, x_r, y_r])
    print('Intersection coordinates: ', intersecting_box)
    intersection_area = calculate_area(intersecting_box)
    union_area = calculate_area(box1) + calculate_area(box2) - intersection_area
    print('IOU/Union: ', intersection_area, '/', union_area)
    iou = intersection_area/union_area

    return iou

  else:
    print('NO intersection')
    return np.array(0)



def get_intersectionv2(box1, box2):
  # v2 also works fine
  x_l = np.maximum(box1[0], box2[0]) # not np.max
  y_l = np.maximum(box1[1], box2[1])

  x_r = np.minimum(box1[2], box2[2])
  y_r = np.minimum(box1[3], box2[3])

  if x_r>=x_l and y_r>=y_l:

    intersecting_box = np.array([x_l, y_l, x_r, y_r])
    print('Intersection coordinates: ', intersecting_box)
    intersection_area = calculate_area(intersecting_box)
    union_area = calculate_area(box1) + calculate_area(box2) - intersection_area
    print('IOU/Union: ', intersection_area, '/', union_area)
    iou = intersection_area/union_area

    return iou

  else:
    print('NO intersection')
    return np.array(0)

print('area1: ', calculate_area(box1))

print('area2: ', calculate_area(box2))

print('area3: ', calculate_area(box3))

print('IOU: ', get_intersection(box1, box2))

print('IOU: ', get_intersection(box1, box3)) ## no intersection

print('IOU: ', get_intersection(box1, box1)) ## perfect overlap

area1:  350
area2:  725
area3:  725
Intersection coordinates:  [20 25 29 35]
IOU/Union:  90 / 985
IOU:  0.09137055837563451
NO intersection
IOU:  0
Intersection coordinates:  [15 10 29 35]
IOU/Union:  350 / 350
IOU:  1.0


In [60]:
boxes = [box1, box2, box3]

n = len(boxes)
iou_matrix = np.zeros((n,n))
for i in range(len(boxes)):
  for j in range(len(boxes)):
    if i!=j:
      iou_matrix[i,j] = get_intersection(boxes[i], boxes[j])

print(iou_matrix)


def get_ioumatrix(boxes):
  n = len(boxes)
  iou_matrix = np.zeros((n,n))
  for i in range(len(boxes)):
    for j in range(len(boxes)):
      if i!=j:
        iou_matrix[i,j] = get_intersection(boxes[i], boxes[j])

  return iou_matrix


Intersection coordinates:  [20 25 29 35]
IOU/Union:  90 / 985
NO intersection
Intersection coordinates:  [20 25 29 35]
IOU/Union:  90 / 985
NO intersection
NO intersection
NO intersection
[[0.         0.09137056 0.        ]
 [0.09137056 0.         0.        ]
 [0.         0.         0.        ]]


In [68]:
### NMS

## putting everything together

def calculate_area(box):
  width = box[2]-box[0]
  height = box[3] - box[1]
  return width*height

def get_intersection(box1, box2):
  x_l = max(box1[0], box2[0])
  y_l = max(box1[1], box2[1])
  x_r = min(box1[2], box2[2])
  y_r = min(box1[3], box2[3])

  if x_r>=x_l and y_r>=y_l:
    intersecting_box = np.array([x_l, y_l, x_r, y_r])
    intersection_area = calculate_area(intersecting_box)
    union_area = calculate_area(box1) + calculate_area(box2) - intersection_area
    iou = intersection_area/union_area
    return iou

  else:
    return np.array(0)




In [112]:
box1 = np.array([15, 10, 29, 35]) # x_l, y_l, x_r, y_r
box2 = np.array([20, 25, 45, 54]) # x_l, y_l, x_r, y_r
box3 = np.array([120, 25, 145, 54]) # x_l, y_l, x_r, y_r
box4 = np.array([18, 10, 29, 35]) # x_l, y_l, x_r, y_r
box5 = np.array([16, 10, 29, 35]) # x_l, y_l, x_r, y_r

scores = np.array([0.9, 0.8, 0.7, 0.85, 0.88]) ## confidence

boxes = [box1, box2, box3, box4, box5]
iou_matrix = get_ioumatrix(boxes)
print(iou_matrix)

boxes_arr = np.stack(boxes)

idxs = np.argsort(scores)[::-1] # [::-1] for reversing
print('sorted idxs: ', idxs)


keep = []
keep.append(idxs[0])
for idx in idxs:
  print(iou_matrix[idx])
  to_remove = (iou_matrix[idx]>0.5).nonzero()[0]

  if to_remove.size>0:
    for idx_r in to_remove:
      print('remove: ', idx_r)

  else:
    print('keep: ', idx )
    keep.append(idx)

print('Final: ', keep)


[[0.         0.09137056 0.         0.78571429 0.92857143]
 [0.09137056 0.         0.         0.0989011  0.09375   ]
 [0.         0.         0.         0.         0.        ]
 [0.78571429 0.0989011  0.         0.         0.84615385]
 [0.92857143 0.09375    0.         0.84615385 0.        ]]
sorted idxs:  [0 4 3 1 2]
[0.         0.09137056 0.         0.78571429 0.92857143]
remove:  3
remove:  4
[0.92857143 0.09375    0.         0.84615385 0.        ]
remove:  0
remove:  3
[0.78571429 0.0989011  0.         0.         0.84615385]
remove:  0
remove:  4
[0.09137056 0.         0.         0.0989011  0.09375   ]
keep:  1
[0. 0. 0. 0. 0.]
keep:  2
Final:  [0, 1, 2]


In [112]:
## Vectorized implementation
## uses broadcasting nicely :)
## source: https://blog.roboflow.com/how-to-code-non-maximum-suppression-nms-in-plain-numpy/

def box_iou_batch(
	boxes_a: np.ndarray, boxes_b: np.ndarray
) -> np.ndarray:

    def box_area(box):
        return (box[2] - box[0]) * (box[3] - box[1])

    area_a = box_area(boxes_a.T)
    area_b = box_area(boxes_b.T)

    top_left = np.maximum(boxes_a[:, None, :2], boxes_b[:, :2])
    bottom_right = np.minimum(boxes_a[:, None, 2:], boxes_b[:, 2:])

    area_inter = np.prod(
    	np.clip(bottom_right - top_left, a_min=0, a_max=None), 2)

    return area_inter / (area_a[:, None] + area_b - area_inter)

boxes_a = np.array([
    [15, 10, 29, 35],
    [20, 25, 45, 54],
    [120, 25, 145, 54]
])

boxes_b = boxes_a

box_iou_batch(boxes_a, boxes_b)